In [1]:
import numpy as np
import re
import pandas as pd
import altair as alt

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = 'assets/'

land_df = pd.read_csv(path + 'combined_tables.csv')
chem_df = pd.read_csv(path + 'ChemDataforJeffOlson.csv')

# Format date field, drop old version

chem_df['Date']= pd.to_datetime(chem_df['VisitDate'])
chem_df.drop('VisitDate', axis=1, inplace=True)

chem_df.sample(7)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,Date
276781,WHEELER (BRUNWK),1,Pelagic,44.70838,-71.64008,Brunswick,AcidLake,1,1038.0,TygonHose,6.00,Reg,GranAlk,NaN,5.27,Y,NaN,NaN,NaN,1983-08-18
73855,ELLIGO,1,Pelagic,44.59381,-72.35560,Greensboro,SpringTP,1,1024.0,Hydrolab,8.01,Reg,DO,NaN,7.80,Y,NaN,NaN,NaN,2014-05-14
176582,NEAL,1,Pelagic,44.49140,-71.69280,Lunenburg,SpringTP,1,1023.0,Hydrolab,2.00,R1,Cond,NaN,25.00,Y,NaN,NaN,NaN,2001-05-17
25005,BRANCH,1,Pelagic,43.08079,-73.01964,Sunderland,AcidLake,1,1530.0,PlasticKemm,1.00,Reg,DNa,NaN,0.46,Y,NaN,NaN,NaN,2010-04-21
191038,PEACHAM,1,Pelagic,44.33060,-72.26000,Peacham,LayMon,1,925.0,Hose,10.00,R1,Chla,NaN,3.38,Y,NaN,NaN,NaN,2010-07-07
85597,FORESTER,1,Pelagic,43.08110,-72.86830,Jamaica,AcidLake,1,1330.0,TygonHose,1.50,R1,VisualTotColor,NaN,15.00,Y,NaN,NaN,NaN,1989-01-25
83461,FOREST (CALAIS),1,Pelagic,44.40750,-72.43890,Calais,Laymon,1,830.0,Secchi,NaN,Reg,Secchi,NaN,7.90,Y,NaN,NaN,NaN,2002-07-06


## Removing Outliers

Based on observation it appears that the data contains many outliers that likely correspond to errors of recording.  For example, some rows show chemical levels that are approximately ten times the overall mean; this likely reflects an extra keystroke during data entry.  The following cell removes rows with measurements that are more than `tol = 3` standard deviations from the mean, for that lake and that particular test.  

**About 6000 rows (2.2 percent), are removed.**  If we assumed normal distribution, we would only expect about 0.3 percent of the data values to be further than 3 standard deviations from the mean.  In other words, there is much more data there than we would expect to see, so it seems error is a likely explanation.

In [3]:
data = []
tol = 3

for lake, lake_df in chem_df.groupby('LakeID'):
    for test, chemlake_df in lake_df.groupby('CharacteristicID'):
        stat_df = chemlake_df.copy()
        lc_mean = np.mean(stat_df.Result)
        lc_std = np.std(stat_df.Result)
        
        add_df = stat_df[ stat_df['Result'].apply(lambda x: abs(x-lc_mean) < tol*lc_std)]
        data.append(add_df)
            
chem_df = pd.concat(data)
chem_df.sort_index(inplace=True)

For reference, these are the definitions of the geographic categories in the `Description` field at the [Vermont DNR website](https://dec.vermont.gov/watershed/lakes-ponds/data-maps/land-cover-maps):

- `Watershed`: the entire watershed.
- `Flowline100ft`: the 100-foot buffer on each side of the tributaries draining into the lake
- `Waterbody100ft`: the 100-foot buffer around the lakeshore
- `Buffer100ftWBFL`: the 100-foot buffer that runs along both sides of the tributary streams into the lake combined with the 100-foot buffer along the lakeshore
- `Buffer250ftWaterbody`: the 250-foot buffer around the lakeshore, the protected shoreland area under the Vermont Shoreland Protection Act

Note that `Shape_Area` only includes land area, not the lake itself.

## Quick tools

These functions make graphs for a fast look at the data.

`snap_chart` has arguments `lake` to identify the specific lake and `chem` for the specific chemical test.  The default data frame is `chem_df`.  It returns a scatter plot of values across the entire history of measurement.

`tre_chart` returns a scatter plot of chlorophyll-a, total phosphorus, and clairty (Secchi) for `lake`.  

`scat_fit_chart` has the same inputs and return as `snap_chart', but adds a line of best fit to the scatter plot. 

In [4]:
def snap_chart(lake, chem, df=chem_df):
    ch = alt.Chart(df[ (df['LakeID'] == lake) & (df['CharacteristicID'] == chem)]).mark_circle().encode(
        alt.X('Date:T'),
        alt.Y('Result:Q')
    )
    return ch

def tre_chart(lake, df=chem_df):
    cdf = df[ df['LakeID'] == lake]
    cdf = cdf[ cdf['CharacteristicID'].apply(lambda x: x in ['TP', 'Chla', 'Secchi'])]
    ch = alt.Chart(cdf).mark_circle().encode(
        alt.X('Date:T'),
        alt.Y('Result:Q'),
        alt.Color('CharacteristicID', scale = alt.Scale( scheme = "lightmulti"))
    )
    return ch

def scat_fit_chart(lake, chem, df=chem_df):
    stat_df = df[ (df.LakeID == lake) & (df.CharacteristicID == chem)]
    tmin = stat_df.Date.min()
    tmax = stat_df.Date.max()
    for i in stat_df.index:
        stat_df.loc[i, 'tdelta']  = (stat_df.loc[i, 'Date'] - tmin).days
    chem_fit = np.polyfit(stat_df.tdelta, stat_df.Result, deg=1)
    fitline = np.poly1d(chem_fit)
    line = pd.DataFrame({
        'Date':[tmin, tmax],
        'Result': fitline([0, (tmax-tmin).days]) 
    })
    scat = alt.Chart(stat_df).mark_circle().encode(
        alt.X('Date:T'),
        alt.Y('Result:Q', title = chem)
    )
    fit = alt.Chart(line).mark_line(color = 'red').encode(
        alt.X('Date:T'),
        alt.Y('Result:Q')
    )
    return scat + fit


In [5]:
scat_fit_chart('GROUT', 'Secchi')

alt.LayerChart(...)

In [6]:
tre_chart('MAIDSTONE')

alt.Chart(...)

In [7]:
snap_chart("CARMI", 'TempC') #+ snap_chart("MAIDSTONE", 'DO')

alt.Chart(...)

In [8]:
lakes_in_land_survey = land_df['LakeID'].unique()
all_lakes = chem_df['LakeID'].unique()

lu_df = chem_df[ chem_df['LakeID'].apply(lambda x: x in lakes_in_land_survey)].copy()

In [9]:
def most_surveyed(n, df=chem_df):
    lake_by_meas = []
    for group, frame in chem_df.groupby('LakeID'):
        lake_by_meas.append((len(frame), group))
    lake_by_meas.sort(reverse=True)
    return df[ df['LakeID'].apply(lambda x: x in [ x[1] for x in lake_by_meas ][:n])]

In [10]:
most_surveyed(10)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,Date
18546,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,Secchi,NaN,Reg,Secchi,NaN,2.30,Y,NaN,NaN,NaN,1982-08-17
18547,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,Cond,NaN,16.00,Y,NaN,NaN,NaN,1982-08-17
18548,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,GranAlk,NaN,0.22,Y,NaN,NaN,NaN,1982-08-17
18549,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,pH,NaN,5.23,Y,NaN,NaN,NaN,1982-08-17
18550,BOURN,1,Pelagic,43.10695,-73.00224,Sunderland,AcidLake,1,1500.0,TygonHose,6.00,Reg,SpectroDisColor,NaN,58.00,Y,NaN,NaN,NaN,1982-08-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281473,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1410.0,Hydrolab,59.83,Reg,TempC,NaN,4.02,Y,NaN,NaN,NaN,2021-07-12
281475,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,DO,NaN,11.78,Y,NaN,NaN,NaN,2021-07-12
281476,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,DO%,NaN,90.10,Y,NaN,NaN,NaN,2021-07-12
281477,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,Westmore,LaymonQC,1,1411.0,Hydrolab,65.23,Reg,pH,NaN,7.26,Y,NaN,NaN,NaN,2021-07-12


## Statistical Summary of Lake Chemistry Data

The following cell creates a dataframe `stat_summary` that records for each lake, and each chemistry test applied to the lake, what the mean, median, and standard deviation is.  Also, the slope and intercept of the line of best fit is provided.  Slope here indicates the *daily* rate of change in the value measured, so (for example) multiply by 365 to get the yearly rate of change.  The intercept is aligned with the earliest date of measurement for the indicated test.

**This cell may take more than a minute to execute.**

In [11]:
data = []

for lake, lake_df in chem_df.groupby('LakeID'):
    for test, chemlake_df in lake_df.groupby('CharacteristicID'):
        if len(set(chemlake_df.Date)) > 1:  # Must be measurements from at least two different days
            stat_df = chemlake_df.copy()
            lc_mean = np.mean(stat_df.Result)
            lc_median = np.median(stat_df.Result)
            lc_std = np.std(stat_df.Result)

            tmin = stat_df.Date.min() 
            tmax = stat_df.Date.max()
            for i in stat_df.index:
                stat_df.loc[i, 'tdelta'] = (stat_df.loc[i, 'Date'] - tmin).days
            lr_coeff = np.polyfit(stat_df.tdelta, stat_df.Result, deg = 1)
            data.append( ( lake, test, lc_mean, lc_median, lc_std, lr_coeff[0], lr_coeff[1], tmin, tmax, len(stat_df) ) )

stat_summary = pd.DataFrame( data, columns = [ 'Lake', 'Test', 'Mean', 'Median', 'Stand_Dev', 'Slope', 'Intercept', 'Begin_Date', 'End_Date', 'N_value' ] ) 

stat_summary.to_csv('assets/statsum.csv')

In [12]:
stat_summary.head(10)

,Lake,Test,Mean,Median,Stand_Dev,Slope,Intercept,Begin_Date,End_Date,N_value
0,ABENAKI,ChlaProbe,7.414167,6.645,6.329228,0.004784,-1.121862,2006-04-18,2012-09-18,12
1,ABENAKI,Cond,68.306667,65.000,9.576323,0.002702,58.146225,1998-04-24,2012-09-18,15
2,ABENAKI,DO,9.008235,9.730,1.601669,-0.000032,9.185627,1991-08-21,2012-09-18,17
3,ABENAKI,DO%,89.013333,91.900,8.046437,-0.002503,98.424657,1998-04-24,2012-09-18,15
4,ABENAKI,RegAlk,28.550000,27.000,4.567549,0.000699,25.536852,1991-08-21,2012-09-18,6
5,ABENAKI,Secchi,2.860000,2.850,0.253772,0.000009,2.827630,1988-04-22,2012-09-18,10
6,ABENAKI,TCa,9.537500,9.530,0.221628,-0.000053,9.649749,1998-04-24,2011-04-27,4
7,ABENAKI,TMg,1.185000,1.180,0.051235,-0.000005,1.194639,1998-04-24,2011-04-27,4
8,ABENAKI,TN,0.233280,0.200,0.082732,0.000006,0.215588,1998-04-24,2012-09-18,5
9,ABENAKI,TP,10.950000,11.000,3.438871,0.000714,8.049348,1988-04-22,2012-09-18,12


The function `trend_sort` projects the `stat_summary` dataframe onto `test_type` chemical test, and returns a version that is sorted according to the values in `Slope`, i.e., according to the rate of change exhibited by lakes with this test.  An optional parameter `min_nvalue` (set at 20) provides a condition on the number of tests a lake must have had in order to be included.

In [13]:
def trend_sort(test_type, min_nvalue=20):
    tdf = stat_summary[ (stat_summary['Test'] == test_type) & (stat_summary['N_value'] >= min_nvalue) ]
    tdf.sort_values('Slope', inplace=True, ascending=False)
    return tdf.set_index('Lake')

In [14]:
# Using trend sort to create dataframes focused on the big three measures of lake health

tp_df = trend_sort('TP')
chla_df = trend_sort('Chla')
secchi_df = trend_sort('Secchi')

# These are the lakes (90) that appear on all three of the above dataframes

common_lakes = list((set(tp_df.index).intersection(set(chla_df.index))).intersection(set(secchi_df.index)))

# Getting basic statistical measures for use in the health metric

tp_mean = tp_df['Mean'].mean()
tp_sd = np.std(tp_df['Mean'])
tp_slope_mean = tp_df['Slope'].mean()
tp_slope_sd = np.std(tp_df['Slope'])
chla_mean = chla_df['Mean'].mean()
chla_sd = np.std(chla_df['Mean'])
chla_slope_mean = chla_df['Slope'].mean()
chla_slope_sd = np.std(chla_df['Slope'])
secchi_mean = secchi_df['Mean'].mean()
secchi_sd = np.std(secchi_df['Mean'])
secchi_slope_mean = secchi_df['Slope'].mean()
secchi_slope_sd = np.std(secchi_df['Slope'])

# Adding columns to record z-scores
# MeanZ gives the lake's z-score with respect to the quantity tested itself
# SlopeZ gives the z-score with respect to the rate of change in that quantity, compared to all rates for that test.

tp_df['MeanZ'] = (tp_df['Mean'] - tp_mean)/tp_sd
tp_df['SlopeZ'] = (tp_df['Slope'] - tp_slope_mean)/tp_slope_sd
chla_df['MeanZ'] = (chla_df['Mean'] - chla_mean)/chla_sd
chla_df['SlopeZ'] = (chla_df['Slope'] - chla_slope_mean)/chla_slope_sd
secchi_df['MeanZ'] = (secchi_df['Mean'] - secchi_mean)/secchi_sd
secchi_df['SlopeZ'] = (secchi_df['Slope'] - secchi_slope_mean)/secchi_slope_sd

In [15]:
tp_df.sample(10)

,Test,Mean,Median,Stand_Dev,Slope,Intercept,Begin_Date,End_Date,N_value,MeanZ,SlopeZ
Lake,,,,,,,,,,,
LONG (SHEFLD),TP,9.904375,9.82,2.080856,0.000178,7.606551,1980-02-25,2020-05-13,48,-0.529453,0.070363
SHELBURNE,TP,106.388083,91.00,55.499522,-0.006664,156.120262,1980-04-11,2019-04-22,193,8.932061,-5.053236
WOODWARD,TP,7.871429,7.69,1.679510,-0.000109,8.825893,1981-04-20,2013-04-30,28,-0.728811,-0.144219
LITTLE (WELLS),TP,15.418994,15.10,3.619206,0.000241,12.500059,1980-04-14,2020-09-14,159,0.011328,0.117802
WATERBURY,TP,13.052000,11.65,6.540355,0.000467,10.460254,1990-05-02,2019-05-31,20,-0.220787,0.287101
GREENWOOD,TP,16.205954,14.00,7.633502,0.000026,15.949352,1980-05-22,2019-09-11,131,0.088500,-0.043360
HARVEYS,TP,11.995258,11.30,3.191895,-0.000250,14.220598,1980-04-25,2019-09-01,329,-0.324414,-0.250399
WOLCOTT,TP,14.470000,13.80,2.684604,-0.000154,15.653481,1982-02-05,2019-05-30,20,-0.081733,-0.177937
PERCH (BENSON),TP,9.918800,9.00,2.626709,-0.000071,10.413710,1983-04-12,2017-08-28,25,-0.528039,-0.116291


## Creating a numerical measure of lake health

We are using a comparative measure; we measure how close a lake is to an "average lake" in the three important tests.  It will not tell us, for example, wwhether all the lakes are experiencing common impacts to their health.

Each lake receives a score that is a non-negative number.  Numbers close to zero indicate that the lake is near the "average lake," and so is considered healthy.

The calculation in more detail is as follows.  For each of the three main tests, we add the z-score for the (mean of the) measure itself to the z-score of the rate of change in the measure experienced by that lake.  This way, if a lake has below average measurements in that quantity, and above average slope (or vice versa) then there is some "correction" happening, i.e., the quantity appears to be trending back towards the mean value.  By adding the scores (one negative, the other positive), there is some "cancellation" towards zero.  Then, the absolute value of these three is added up.  The result is returned as the health score.

In [16]:
def health_score(lake):
    tp_score = tp_df.loc[lake, 'MeanZ'] + tp_df.loc[lake, 'SlopeZ']
    chla_score = chla_df.loc[lake, 'MeanZ'] + chla_df.loc[lake, 'SlopeZ']
    secchi_score = secchi_df.loc[lake, 'MeanZ'] + secchi_df.loc[lake, 'SlopeZ']
    return abs(tp_score) + abs(chla_score) + abs(secchi_score)

In [17]:
health_score_df = pd.DataFrame( {'Lake' : common_lakes, 'Health_Score' : [ health_score(lake) for lake in common_lakes]})

health_score_save_path = 'assets/health_metric.csv'

health_score_df.to_csv(health_score_save_path)

health_score_df.sort_values('Health_Score')

,Lake,Health_Score
7,DUNMORE,0.089848
84,GREEN RIVER,0.169875
63,SALEM,0.286378
47,JOES (DANVLL),0.374875
86,BURR (SUDBRY),0.425024
...,...,...
62,PENSIONER,5.267679
16,HIGH (SUDBRY),7.091692
70,CHAMP-SOUTH LAKE,7.978622
18,SHELBURNE,10.045104


In [18]:
tre_chart('SALEM')

alt.Chart(...)

In [19]:
tre_chart('SHELBURNE')

alt.Chart(...)

In [20]:
data = []
 
for lake in common_lakes:
    data.append( [ lake, 365 * tp_df.loc[lake, 'Slope'], 365 * chla_df.loc[lake, 'Slope'], 365 * secchi_df.loc[lake, 'Slope']])
    
bigthree_df = pd.DataFrame(data, columns = ['Lake', 'dPhos', 'dChla', 'dSecchi']).set_index('Lake')

In [21]:
bigthree_df.sample(10)

,dPhos,dChla,dSecchi
Lake,,,
ECHO (CHARTN),0.075773,-0.009426,-0.004399
NORTH MONTPELIER,0.085177,-0.066601,0.005211
PERCH (BENSON),-0.026033,0.129046,0.046444
CARMI,0.185081,-0.073855,0.009322
HALLS,0.095964,0.061611,0.008185
HIGH (SUDBRY),0.418670,-1.461015,0.049963
RESCUE,0.088765,0.007000,-0.012671
BLISS,0.066371,0.370027,-0.000393
CHIPMAN,0.290233,0.064191,-0.005615


In [22]:
bt_chart = alt.Chart(bigthree_df).mark_circle().encode(
    alt.Size('dSecchi:Q', scale = alt.Scale( scheme = "lightmulti")),
    alt.Color('dSecchi:Q', scale = alt.Scale( scheme = "lightmulti")),
    alt.Y('dPhos:Q'),
    alt.X('dChla:Q')
).properties(width=800, height=500)

bt_chart

alt.Chart(...)

In [23]:
bigthree_df.drop('ADAMANT', inplace=True)

## Lake Health Metric

We put forward the following metric for lake health.  We look at mean total phosphorus, chlorophyl-a, and Secchi water clarity scores, and also their trends, for each lake.  Then we look at each lake's z-score  (in absolute value) with respect to these, and take the sum.  This metric measures (1) how far a lake's mean measures are from the other lakes, and (2) to what degree the trend in these measures diverges from that in other lakes.